In [129]:
# Set your API key here:

# import os
# os.environ['OPENAI_API_KEY'] = ''

## LLMChain

In [130]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

prompt_template = "Write a programming joke, make the joke about the following language: {language}?"

llm = ChatOpenAI()
llm_chain = LLMChain(
    llm=llm,
    prompt= ChatPromptTemplate.from_template(prompt_template)
)
llm_chain("Php")

{'language': 'Php',
 'text': "Why did the PHP developer go broke?\n\nBecause he couldn't find his $cents!"}

## LCEL

In [131]:
chain = ChatPromptTemplate.from_template(prompt_template) | ChatOpenAI(temperature=0)
chain.invoke({
    "language": "JavaScript"
})

AIMessage(content='Why did the JavaScript developer go broke?\n\nBecause he lost all his prototypes!')

------

## Standard Ways of Only LLM Based Chains

### LLMChain

In [132]:
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

In [133]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
synopsis_template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.
Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
synopsis_prompt_template = PromptTemplate(input_variables=["title", "era"], template=synopsis_template)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt_template, output_key="synopsis")

In [134]:
# This is an LLMChain to write a review of a play given a synopsis.
review_template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
review_prompt_template = PromptTemplate(input_variables=["synopsis"], template=review_template)
review_chain = LLMChain(llm=llm, prompt=review_prompt_template, output_key="review")

In [135]:
llm = ChatOpenAI(temperature=.7)
social_template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, the date,time and location, 
the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
social_prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=social_template)
social_chain = LLMChain(llm=llm, prompt=social_prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': '🌅✨ Don\'t miss the breathtaking tragedy that is "Tragedy at Sunset on the Beach"! Transport yourself to Victorian England, where forbidden love and heart-wrenching sacrifices take center stage. 🎭💔\n\n📅 Date: December 25th\n⏰ Time: 8pm PST\n📍 Location: Theater in the Park\n\nSynopsis: In a dreary coastal town, Amelia and Edward find solace in each other\'s arms, defying societal norms and risking everything for love. But their secret rendezvous on the desolate beach at sunset leads to a tragic duel that will leave you breathless. Prepare for a journey of love, loyalty, and sacrifice.\n\nReview: The New York Times praises the exceptional performances, evocative stage design, and captivating storytelling. This play will immerse you in the intricate world of Victorian England and leave you reflecting on the timeless struggle b

### LCEL

In [70]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter

In [149]:
snyopsis_chain = (
    synopsis_prompt_template | ChatOpenAI() |  StrOutputParser() | { "synopsis": RunnablePassthrough(), "title": itemgetter("title"), "era": itemgetter("era") } 
)

review_chain = ({
        "review":  review_prompt_template | ChatOpenAI() | StrOutputParser() | RunnablePassthrough(),
        "synopsis": itemgetter("synopsis"),
        "title": itemgetter("title"),
        "era": itemgetter("era")
})


social_chain = (
    RunnableParallel(
        synopsis=itemgetter("synopsis"),
        review=itemgetter("review"),
        time=lambda x: "December 25th, 8pm PST",
        location=lambda x: 'London'
    ) | 
    social_prompt_template | ChatOpenAI() | StrOutputParser() | RunnablePassthrough()
)

sequential_chain = (
    synopsis_chain | review_chain | { "social_media_post": social_chain, 
                                      "synopsis": itemgetter("synopsis"), 
                                      "review": itemgetter("review"),
                                      "title": itemgetter("title"),
                                      "era": itemgetter("era")}
                                      )

In [150]:
lcel_chain = sequential_chain.invoke({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})

In [151]:
print(lcel_chain.keys())

dict_keys(['social_media_post', 'synopsis', 'review', 'title', 'era'])


In [152]:
print(lcel_chain)

{'social_media_post': '🌅 Introducing "Tragedy at Sunset on the Beach" 🌅\n\nStep back in time to Victorian England and witness a tale of forbidden love, betrayal, and the devastating consequences of societal constraints. 🎭✨\n\n🗓️ Date & Time: December 25th, 8pm PST\n📍 Location: London\n\nJoin Cecilia and Robert as their paths intertwine on a secluded beach, and a connection, both immediate and undeniable, forms between them. 💔\n\nTheir secret affair blossoms under the veil of darkness, but their happiness is short-lived when Cecilia\'s domineering family discovers their forbidden romance. As societal pressures mount and emotions reach their boiling point, the stage is set for a heart-wrenching climax. 😢\n\nCritics rave about "Tragedy at Sunset on the Beach," calling it a mesmerizing theatrical experience that captures the essence of love and societal constraints in Victorian England. 🌹\n\nImmerse yourself in the opulent world of the upper class and witness the collision of love and duty